# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [43]:
# import libraries
import nltk
import numpy as np
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sqlalchemy import create_engine
import re
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /Users/pepe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/pepe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/pepe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
# load data from database
engine = create_engine('sqlite:///db_disaster_response.db')
df = pd.read_sql_table('disaster_messages', con=engine)
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
X = df["message"]
y = df[df.columns[4:]]# We exclude the first 4 columns (id, message, original, genre)
category_names = y.columns

In [80]:
y.shape, X.shape

((26216, 36), (26216,))

### 2. Write a tokenization function to process your text data

In [81]:
# def tokenize(text):
#     """
#     It receives a text and return clear tokens from the sentence
#     """
#     tokens = word_tokenize(text)
# #     tokens = [t for t in tokens if t not in stopwords.words('english')] # Remove stopwords
#     lemmatizer = WordNetLemmatizer()
#     clean_tokens = []
#     for token in tokens:
#         clean_token = lemmatizer.lemmatize(token).lower().strip()
#         clean_tokens.append(clean_token)
#     return clean_tokens

# # Inspired from here: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/#wordnetlemmatizer
def tokenize(text):
    """tokenize and transform input text. Return cleaned text"""
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # take out all punctuation while tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    # lemmatize as shown in the lesson
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
    return clean_tokens

# def tokenize(text):
#     """
#     Tokenizes text data
#     Args:
#     text str: Messages as text data
#     Returns:
#     words list: Processed text after normalizing, tokenizing and lemmatizing
#     """
#     # Normalize text
#     text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
#     # tokenize text
#     words = word_tokenize(text)
    
#     # remove stop words
#     stopwords_ = stopwords.words("english")
#     words = [word for word in words if word not in stopwords_]
    
#     # extract root form of words
#     words = [WordNetLemmatizer().lemmatize(word, pos='v') for word in words]

#     return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [82]:
# We need to verify that the words are being tokenized (This willl not be included in the final script) 

# words_vector = CountVectorizer(tokenizer=tokenize)
# X_vectorized = words_vector.fit_transform(X)

# keys, values = [], []
# for k, v in words_vector.vocabulary_.items():
#     keys.append(k)
#     values.append(v)

# df_words = pd.DataFrame.from_dict({'words': keys, 'count': values})
# df_words
# plt.figure(figsize=(16, 6))
# sns.barplot(x="words", y="count", data=df_words)

In [90]:
moc = MultiOutputClassifier(RandomForestClassifier())

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', moc)
])

# parameters = {'clf__estimator__max_depth': [10, 50, None],
#               'clf__estimator__min_samples_leaf': [2, 5, 10]}

# cv = GridSearchCV(pipeline, parameters)

In [86]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1ae8081c20>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [87]:
# We split the test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20972,), (5244,), (20972, 36), (5244, 36))

In [88]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [89]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
def performance_results(model, X_test, y_test):
    '''
    Function to generate classification report on the model
    @param model: the model 
    @param X_test: vector containing features
    @param y_test: vector containing the predictions
    Output: Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a266b7950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                 

In [13]:
parameters =  {'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4],
              'tfidf__use_idf': (True, False)} 

cv = GridSearchCV(pipeline, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train, y_train)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/pepe/anaconda3/envs/python3_new/lib/python3.7/site-packages/joblib/parallel.py", line 1007, in __call__
    while self.dispatch_one_batch(iterator):
  File "/Users/pepe/anaconda3/envs/python3_new/lib/python3.7/site-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._dispatch(tasks)
  File "/Users/pepe/anaconda3/envs/python3_new/lib/python3.7/site-packages/joblib/parallel.py", line 754, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/Users/pepe/anaconda3/envs/python3_new/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 209, in apply_async
    result = ImmediateResult(func)
  File "/Users/pepe/anaconda3/envs/python3_new/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 590, in __init__
    self.results = batch()
  File "/Users/pepe/anaconda3/envs/python3_new/lib/python3.7/site-packages/joblib/parallel.py", line 256, in __call__
    for func, args, kwargs in sel

TypeError: can only concatenate str (not "list") to str

In [ ]:
performance_results(cv, X_test, y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])



In [ ]:
pipeline_2.fit(X_train, y_train)
performance_results(pipeline_2, X_test, y_test)

In [ ]:
parameters_2 = {'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2],
              'tfidf__use_idf': (True, False), }

In [ ]:
cv_2 = GridSearchCV(pipeline_2, param_grid=parameters_2)
cv_2

### 9. Export your model as a pickle file

In [ ]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(cv_2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.